# txt2xml

In [3]:
import os, sys
import glob
from PIL import Image
 
# VEDAI 图像存储位置
src_img_dir = r"G:\zhenyu\data\1x1_all_ng_img"
# VEDAI 图像的 ground truth 的 txt 文件存放位置
src_txt_dir = r"G:\zhenyu\data\1x1_label\1x1_all_ng_txt\txt"
src_xml_dir = r"G:\zhenyu\VOC2007\Annotations"
 
img_Lists = glob.glob(src_img_dir + '/*.bmp')
 
img_basenames = [] # e.g. 100.jpg
for item in img_Lists:
    img_basenames.append(os.path.basename(item))
 
img_names = [] # e.g. 100
for item in img_basenames:
    temp1, temp2 = os.path.splitext(item)
    img_names.append(temp1)
 
for img in img_names:
    im = Image.open((src_img_dir + '/' + img + '.bmp'))
    width, height = im.size
 
    # open the crospronding txt file
    gt = open(src_txt_dir + '/' + img + '.txt').read().splitlines()
    #gt = open(src_txt_dir + '/gt_' + img + '.txt').read().splitlines()
 
    # write in xml file
    #os.mknod(src_xml_dir + '/' + img + '.xml')
    xml_file = open((src_xml_dir + '/' + img + '.xml'), 'w')
    xml_file.write('<annotation>\n')
    xml_file.write('    <folder>VOC2007</folder>\n')
    xml_file.write('    <filename>' + str(img) + '.bmp' + '</filename>\n')
    xml_file.write('    <size>\n')
    xml_file.write('        <width>' + str(width) + '</width>\n')
    xml_file.write('        <height>' + str(height) + '</height>\n')
    xml_file.write('        <depth>3</depth>\n')
    xml_file.write('    </size>\n')
 
    # write the region of image on xml file
    for img_each_label in gt:
        spt = img_each_label.split(' ') #这里如果txt里面是以逗号‘，’隔开的，那么就改为spt = img_each_label.split(',')。
        xml_file.write('    <object>\n')
        xml_file.write('        <name>' + str(spt[0]) + '</name>\n')
        xml_file.write('        <pose>Unspecified</pose>\n')
        xml_file.write('        <truncated>0</truncated>\n')
        xml_file.write('        <difficult>0</difficult>\n')
        xml_file.write('        <bndbox>\n')
        xml_file.write('            <xmin>' + str(spt[1]) + '</xmin>\n')
        xml_file.write('            <ymin>' + str(spt[2]) + '</ymin>\n')
        xml_file.write('            <xmax>' + str(spt[3]) + '</xmax>\n')
        xml_file.write('            <ymax>' + str(spt[4]) + '</ymax>\n')
        xml_file.write('        </bndbox>\n')
        xml_file.write('    </object>\n')
 
    xml_file.write('</annotation>')


## split train/test

In [ ]:
import os
import random
 
trainval_percent = 0.85
train_percent = 0.9
xmlfilepath=r'G:\zhenyu\voc\Annotations'
txtsavepath=r"G:\zhenyu\voc\ImageSets\Main"
total_xml = os.listdir(xmlfilepath)
 
num=len(total_xml)
num_list=range(num)
tv=int(num*trainval_percent)
tr=int(tv*train_percent)
trainval= random.sample(list,tv)
train=random.sample(trainval,tr)
 
ftrainval = open(r'G:\zhenyu\voc\ImageSets\Main\trainval.txt', 'w')
ftest = open(r'G:\zhenyu\voc\ImageSets\Main\test.txt', 'w')
ftrain = open(r'G:\zhenyu\voc\ImageSets\Main\train.txt', 'w')
fval = open(r'G:\zhenyu\voc\ImageSets\Main\val.txt', 'w')
 
for i  in num_list:
    name=total_xml[i][:-4]+'\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)
 
ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

## readxml

In [ ]:
import json, os, sys,random
import cv2
import cv2 as cv
import numpy as np
import xml.etree.ElementTree as ET


def load_xmlfile(xml_dir,xml_name,img_dir):
    """
    Load image and bounding boxes info from XML file in the PASCAL VOC
    format.
    """

    imgname = xml_name.replace(".xml", ".bmp")
    xml_name=os.path.join(xml_dir, xml_name)
    print(xml_name)
    tree = ET.parse(xml_name)
    objs = tree.findall('object')
    num_objs = len(objs)
    print(xml_name,os.path.join(img_dir, imgname))
    image = cv2.imread(os.path.join(img_dir, imgname),1)
    #image = gammer(image,0.5)
    boxes = np.zeros((num_objs, 4), dtype=np.uint16)
    gt_classes = list()
    rect_list = list()
    # Load object bounding boxes into a data frame.
    for ix, obj in enumerate(objs):
        bbox = obj.find('bndbox')
        # Make pixel indexes 0-based
        #print (x1)
        x1 = float(bbox.find('xmin').text)
        y1 = float(bbox.find('ymin').text)
        x2 = float(bbox.find('xmax').text)
        y2 = float(bbox.find('ymax').text)
        cls = obj.find('name').text.strip()
        #cls = class_to_ind[obj.find('name').text.strip()]
        rect = [x1, y1, x2, y2]
        cv2.rectangle(image,(int(x1),int(y1)),(int(x2),int(y2)),(0,128,0),1)
        font = cv.FONT_HERSHEY_SIMPLEX
        cv.putText(image, str(cls), (int(x1),int(y1)), font, 1, (255, 255, 255), 1)
        rect_list.append(rect)
        gt_classes.append(cls)
    savedir= "./gt_d/"
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    cv2.imwrite(savedir+imgname,image)
    #cv2.waitKey(0)
    return rect_list, gt_classes


if __name__ == '__main__':
    xml_dir = "G:\zhenyu\VOC2007\Annotations"
    xml_name = "SK_QFN_DY_P01_FM_20190318_131447-crop_03404_00663_03711_00968.xml"
    img_dir = "G:\zhenyu\VOC2007\JPEGImages"
    load_xmlfile(xml_dir, xml_name, img_dir)

In [1]:
import os
import random
import shutil
train_percent = 0.8

imgfilepath = r'G:\zhenyu\VOC2007\JPEGImages'
xmlfilepath=r'G:\zhenyu\VOC2007\Annotations'

train_xml_path=r"G:\zhenyu\VOC2007\train\ann"
train_img_path = r"G:\zhenyu\VOC2007\train\img"
test_xml_path=r"G:\zhenyu\VOC2007\test\ann"
test_img_path=r"G:\zhenyu\VOC2007\test\img"

total_xml = os.listdir(xmlfilepath)

num=len(total_xml)

num_list=range(num)

tr =int(num * train_percent)

train = random.sample(num_list,tr)

for i in range(num):
    if i in train:
        shutil.move(os.path.join(xmlfilepath,total_xml[i]), train_xml_path)
        shutil.move(os.path.join(imgfilepath,os.path.splitext(total_xml[i])[0] + ".bmp" ), train_img_path)
    else:
        shutil.move(os.path.join(xmlfilepath,total_xml[i]), test_xml_path)
        shutil.move(os.path.join(imgfilepath, os.path.splitext(total_xml[i])[0] + ".bmp"), test_img_path)


# ftrainval = open(r'G:\zhenyu\voc\ImageSets\Main\trainval.txt', 'w')
# ftest = open(r'G:\zhenyu\voc\ImageSets\Main\test.txt', 'w')
# ftrain = open(r'G:\zhenyu\voc\ImageSets\Main\train.txt', 'w')
# fval = open(r'G:\zhenyu\voc\ImageSets\Main\val.txt', 'w')
 
# for i  in list:
#     name=total_xml[i][:-4]+'\n'
#     if i in trainval:
#         ftrainval.write(name)
#         if i in train:
#             ftrain.write(name)
#         else:
#             fval.write(name)
#     else:
#         ftest.write(name)
 
# ftrainval.close()
# ftrain.close()
# fval.close()
# ftest.close()